<a href="https://colab.research.google.com/github/Byeon-MJ/DL_Computer_Vision/blob/main/Face_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Tracking


## Tracking 하는 이유?
- Tracking이 Detection보다 처리속도가 빠르다
- 따라서 제일 처음에만 Detection하고 이후에는 Tracking을 해서 부드럽게 Tracking 하도록 구현

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import cv2
import time

## Object Tracker mode 설정

In [ ]:
# csrt
# tracker = cv2.TrackerCSRT_create()
# kcf
tracker = cv2.TrackerKCF_create()
# boosting
# tracker = cv2.TrackerBoosting_create()
# mil
# tracker = cv2.TrackerMIL_create()
# tld
# tracker = cv2.TrackerTLD_create()
# medianflow
# tracker = cv2.TrackerMedianFlow_create()
# mosse
# tracker = cv2.TrackerMOSSE_create()

In [ ]:
# 얼굴을 인식하는 haarcascade feature 파일
face_cascade_name = '/content/gdrive/MyDrive/CV/Face Detection/data/haarcascades/haarcascade_frontalface_alt.xml'
# face_cascade 객체 생성
face_cascade = cv2.CascadeClassifier()

if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('### Error loading face cascade ###')
    exit(0)

In [ ]:
# 원본 동영상
file_name = '/content/gdrive/MyDrive/CV/Face Tracking/mytracking.mp4'
# Tracking 된 output 동영상
output_name = 'face_tracking_video.mp4'

# 변수 초기화
frame_count = 0
detected = False
frame_mode = 'Tracking'
elapsed_time = 0

In [ ]:
# 원본 동영상 로드
cap = cv2.VideoCapture(file_name)

# 원본 동영상 오픈 여부 확인
if not cap.isOpened:
    print('--(!) Error opening video capture')
    exit(0)

writer = None

# trackers 객체 생성
trackers = cv2.legacy.MultiTracker_create()

In [ ]:
while True:
    # 원본 동영상에서 frame을 읽는다
    ret, frame = cap.read()

    # 원본 동영상에서 더 이상 frame을 읽지 못하면 Exit
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print(f'elapsed time {elapsed_time:.3f} seconds')
        break
    
    start_time = time.time()
    frame_count += 1

    if detected:
        frame_mode = 'Tracking'
        # trackers 객체가 tracking 하여 변경된 boxing 영역 위치 좌표를 업데이트
        (success, boxes) = trackers.update(frame)
        for box in boxes:
            